In [11]:
import astropy.io.fits as fits
from astropy.stats import sigma_clip
from astropy.table import Table
from astropy.time import Time
import glob
import gzip
import io
import matplotlib.pylab as plt
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.colors as colors
import numpy as np
import os
import pandas as pd
from progressbar import progressbar
from scipy import stats

#### Declaring all auxiliary functions

In [12]:
def make_three_plots(data_plot, name, labels, epoch="", pdf=""):
    fig, axes = plt.subplots(1,3, figsize=(15,5))
    norm = colors.AsinhNorm(vmin=data_plot[0].min(), vmax=data_plot[0].max())
    for i, ax in enumerate(axes.flat):
        if i < 2:
            im = ax.imshow(data_plot[i], norm=norm)
        else:
            im = ax.imshow(data_plot[i])
        if i == 1:
            plt.colorbar(im, ax=axes.ravel().tolist())
        ax.set_title(f"{labels[i]}: {name} ({epoch})")
        ax.axvline(63/2., color='red', alpha=0.2)
        ax.axhline(63/2., color='red', alpha=0.2)
    if pdf != "":
        pdf.savefig(fig)
    plt.close()

In [13]:
def read_bytes_image(bytes_str):
    hdu_list = fits.open(gzip.open(io.BytesIO(bytes_str)))
    primary_hdu = hdu_list[0]
    return primary_hdu.data

def stack_epochs(data_epochs, obj_id, labels, pdf=""):
    stacked = [[], [], []]
    for i in range(len(data_epochs)):
        stacked[i] = np.median(data_epochs[i], axis=0)
        # print('Hi', type(stacked[i]), stacked[i].shape)
        # print(len(stacked[i][~np.isnan(stacked[i])]))

    return stacked

In [14]:
def resampling_alberto(input_data, bad_fmt, random_inside=[]):
    
    if len(random_inside)==0:
        if bad_fmt is None:
            cleaned_data = input_data[np.where(input_data != None)]
        elif bad_fmt is np.nan:
            cleaned_data = input_data[~np.isnan(input_data)]
        else:
            raise Exception("Invalid bad format for resampling (None or np.nan)")
        output = np.copy(input_data).reshape(-1)
    
        data_1d = cleaned_data.reshape(-1)
        gkde_obj = stats.gaussian_kde(data_1d.astype(float), 0.1) # bw_method='scott')
        x_pts = np.linspace(0, data_1d.max(), 1000)
        estimated_pdf = gkde_obj.evaluate(x_pts)
        random_inside = gkde_obj.resample(5000)[0]
        # plt.figure()
        # plt.hist(data_1d, bins=100, density=True)  # bins='scott'
        # plt.hist(random_inside, bins=100, density=True)
        # plt.plot(x_pts, estimated_pdf, label="kde estimated PDF", color="r")
        # plt.xlim(0,1000)
        # plt.show()
    
        # xy_min = [0, 0]
        # xy_max = [data_1d.max(), 1.05*estimated_pdf.max()]
        # random_xy = np.random.uniform(low=xy_min, high=xy_max, size=(20000,2))
        # random_pdf = gkde_obj.evaluate(random_xy[:,0])
        # inside_pdf = random_xy[:,1] < random_pdf
        # # plt.scatter(random_xy[:,0][inside_pdf], random_xy[:,1][inside_pdf], s=1,color='red')
        # # plt.scatter(random_xy[:,0][~inside_pdf], random_xy[:,1][~inside_pdf], s=1,color='blue')
        # # plt.plot(x_pts, estimated_pdf, label="kde estimated PDF", color="r")
        # # plt.show()
        # random_inside = random_xy[:,0][inside_pdf]
    
    if bad_fmt is None:
        num_invalids = len(input_data[np.where(input_data == None)])
        output[np.where(output == None)] = np.random.choice(random_inside, size=num_invalids)
    elif bad_fmt is np.nan:
        num_invalids = len(input_data[np.isnan(input_data)])
        output[np.isnan(output)] = np.random.choice(random_inside, size=num_invalids)
    
    return output.reshape((63,63))

def replace_clipped(clipped_sci, clipped_temp, invalid):

    # TRY TO FIX IT TOMORROW (4-5 TIMES FASTER)
    final_sci, final_temp = np.copy(clipped_sci).reshape(-1), np.copy(clipped_temp).reshape(-1)
    num_invalids = len(clipped_sci[np.isnan(clipped_sci)])
    dist_sci = np.random.normal(np.median(clipped_sci[~np.isnan(clipped_sci)]),
                                np.std(clipped_sci[~np.isnan(clipped_sci)]), num_invalids)
    final_sci[np.isnan(final_sci)] = dist_sci
    num_invalids = len(clipped_temp[np.isnan(clipped_temp)])
    dist_temp = np.random.normal(np.median(clipped_temp[~np.isnan(clipped_temp)]),
                                 np.std(clipped_temp[~np.isnan(clipped_temp)]), num_invalids)
    final_temp[np.isnan(final_temp)] = dist_temp
    # print(final_sci.shape, final_temp.shape)
    
    return final_sci.reshape((63,63)), final_temp.reshape((63,63))

def plot_clipped_and_resampled(clipped, final, obj_id, lab, pdf=""):
    fig, axes = plt.subplots(1,2, figsize=(10,4))
    im = axes[0].imshow(clipped)
    axes[0].set_title(f"{lab}: {obj_id} (sig-clip)")
    axes[1].imshow(final)
    axes[1].set_title(f"{lab}: {obj_id} (resampled)")
    fig.colorbar(im, ax=axes.ravel().tolist())
    if pdf != "":
        pdf.savefig(fig)
    plt.close()

#### Declaring table with all the objects

In [15]:
columns_63_63 = [[np.ones((63, 63))]]*7
nms = ['stacked_science', 'stacked_template', 'stacked_difference', 'clipped_science',
       'clipped_template', 'resampled_science', 'resampled_template']
final_table = Table(columns_63_63, names=nms)
final_table.add_column(['example'], name='obj_id', index=0)
final_table.add_column([1], name='n_epochs', index=1)
final_table.remove_row(0)

#### Taking everything from Simbad (Priscila selection vs. Emille YSOs)

In [16]:
data_folder = "/media3/CRP7/hosts/data/"
priscila_simbad = data_folder + 'EXTRAGALACTIC/extragal_nocathost.pkl'
priscila_simbad = pd.read_pickle(priscila_simbad)[:2109]
priscila_classes = set(priscila_simbad['v:classification_best'])
# simbad_yso = priscila_simbad[priscila_simbad['v:classification_best']=='YSO']
print(priscila_classes)

{'BLLac', 'AGN_Candidate', 'SN*_Candidate', 'Candidate_SN*', 'Blazar', 'Inexistent', 'QSO', 'QSO_Candidate', 'AGN', 'SN', 'Seyfert_1', 'Seyfert_2', 'Blazar_Candidate', 'Seyfert', 'SN candidate', None}


In [17]:
### 1) Using the *** obj_info *** folder

# classes = os.listdir(data_folder + 'SIMBAD/Apr2023/obj_info')
# classes = [x.split('=')[1] for x in classes if '.pkl' not in x]
# print(sorted(classes))

class_name = "Seyfert"  # add YSO_Candidate later...
simbad_folder = f'{data_folder}/SIMBAD/Apr2023/obj_info/finkclass={class_name}/'
# objects_YSO = pd.read_pickle(simbad_folder + '../objects_YSO.pkl')
# print(objects_YSO) # 2246 items (1123 unique IDs)

# Let's try to reproduce this object list, as a double check
dict_mjds, dict_sci_img, dict_temp_img, dict_diff_img = {}, {}, {}, {}
# for pkl_file in progressbar(sorted(os.listdir(simbad_folder))[:1]): # make all later...
for pkl_file in sorted(os.listdir(simbad_folder)):
    pkl_data = pd.read_pickle(simbad_folder + pkl_file) # this takes long
    for idx in progressbar(range(len(pkl_data))):
        obj_id = pkl_data.iloc[idx]['i:objectId']
        mjd = pkl_data.iloc[idx]['i:jd']
        sci_img = np.array(pkl_data.iloc[idx]['b:cutoutScience_stampData'])
        temp_img = np.array(pkl_data.iloc[idx]['b:cutoutTemplate_stampData'])
        diff_img = np.array(pkl_data.iloc[idx]['b:cutoutDifference_stampData'])
        sci_img[np.where(sci_img == None)] = np.nan
        temp_img[np.where(temp_img == None)] = np.nan
        diff_img[np.where(diff_img == None)] = np.nan
        # print(obj_id, mjd, sci_img.shape, temp_img.shape, diff_img.shape)

        sci_img = sci_img.astype(float)
        temp_img = temp_img.astype(float)
        diff_img = diff_img.astype(float)
        if len(sci_img[~np.isnan(sci_img)]) != 0:
            if obj_id not in dict_mjds.keys():
                dict_mjds[obj_id] = [mjd]
                dict_sci_img[obj_id] = [sci_img]
                dict_temp_img[obj_id] = [temp_img]
                dict_diff_img[obj_id] = [diff_img]
            else:
                dict_mjds[obj_id].append(mjd)
                dict_sci_img[obj_id].append(sci_img)
                dict_temp_img[obj_id].append(temp_img)
                dict_diff_img[obj_id].append(diff_img)
    
    # print(idx)
    print(pkl_file, pkl_file.split('_')[-1].split('.')[0], end=' ')
    number_of_imgs = sum(len(dict_mjds[key]) for key in dict_mjds.keys())
    print(number_of_imgs, len(dict_mjds.keys()))

100% (1584 of 1584) |####################| Elapsed Time: 0:00:07 Time:  0:00:07


SIMBAD_Apr2023_batch_0.pickle 0 1584 5


100% (598 of 598) |######################| Elapsed Time: 0:00:02 Time:  0:00:02


SIMBAD_Apr2023_batch_1.pickle 1 2182 10


100% (397 of 397) |######################| Elapsed Time: 0:00:01 Time:  0:00:01


SIMBAD_Apr2023_batch_2.pickle 2 2579 15


100% (234 of 234) |######################| Elapsed Time: 0:00:01 Time:  0:00:01


SIMBAD_Apr2023_batch_3.pickle 3 2813 18


In [18]:
### 2) Using the *** ftransfer_ztf_2023-08-31_137151 *** folder (DON'T USE!!!)

# simbad_folder = data_folder + 'SIMBAD/Apr2023/ftransfer_ztf_2023-08-31_137151/'
# classes = [x.split('=')[1] for x in os.listdir(simbad_folder) if '.pkl' not in x]
# # print(sorted(classes))

# class_name = "YSO"  # add YSO_Candidate later...
# simbad_folder += f'finkclass={class_name}/'

# # There are 1271 parquet files, containing 1125 unique IDs (only 2671 imgs)
# dict_mjds, dict_sci_img, dict_temp_img, dict_diff_img = {}, {}, {}, {}
# for parquet_file in progressbar(sorted(os.listdir(simbad_folder))):
#     parquet_data = pd.read_parquet(simbad_folder + parquet_file)
#     for idx in range(len(parquet_data)):
#         obj_id = parquet_data.iloc[idx]['objectId']
#         mjd = Time(parquet_data.iloc[idx]['timestamp']).to_value('mjd')
#         sci_img = read_bytes_image(parquet_data.iloc[idx]['cutoutScience']['stampData'])
#         temp_img = read_bytes_image(parquet_data.iloc[idx]['cutoutTemplate']['stampData'])
#         diff_img = read_bytes_image(parquet_data.iloc[idx]['cutoutDifference']['stampData'])
        
#         if obj_id not in dict_mjds.keys():
#             dict_mjds[obj_id] = [mjd]
#             dict_sci_img[obj_id] = [sci_img]
#             dict_temp_img[obj_id] = [temp_img]
#             dict_diff_img[obj_id] = [diff_img]
#         else:
#             dict_mjds[obj_id].append(mjd)
#             dict_sci_img[obj_id].append(sci_img)
#             dict_temp_img[obj_id].append(temp_img)
#             dict_diff_img[obj_id].append(diff_img)
            
# # print(dict_mjds, dict_sci_img, dict_temp_img, dict_diff_img)

# number_of_ids = [len(dict_mjds[key]) for key in dict_mjds.keys()]
# print(len(number_of_ids), np.sum(number_of_ids))
# print(number_of_ids)

#### Stacking images for each of the 1123(5) objects

In [19]:
labels = ['Science', 'Template', 'Difference']
pdf_dir = f'{data_folder}/SIMBAD/images_stack_sigclip/finkclass={class_name}/'
if not os.path.isdir(pdf_dir): os.mkdir(pdf_dir)

# idx = -1
exception_ids = []
for key, value in progressbar(dict_mjds.items()):

    # idx += 1
    # if idx < 977:
    #     continue
    
    # print(key, len(value))
    data_epochs = [dict_sci_img[key], dict_temp_img[key], dict_diff_img[key]]
    # print(len(data_epochs[0]), 'epochs:', data_epochs[0][0].shape)
    pdf = PdfPages(f'{pdf_dir}/{key}_plots.pdf')
    # pdf = ""
    try:
        stacked = stack_epochs(data_epochs, key, labels, pdf)
        stacked[0] = resampling_alberto(stacked[0], np.nan)
        stacked[1] = resampling_alberto(stacked[1], np.nan)
        stacked[2] = resampling_alberto(stacked[2], np.nan)
    except Exception:
        # print(f'Object {key} skipped, it has only strange values...')
        exception_ids.append(key)
        pdf.close()
        continue
    make_three_plots(stacked, key, labels, "median", pdf=pdf)
        
    clipped_sci = sigma_clip(stacked[0], sigma=3, maxiters=10)  # 3 or 5
    clipped_temp = sigma_clip(stacked[1], sigma=3, maxiters=10)  # 3 or 5
    clipped_sci, clipped_temp = clipped_sci.filled(np.nan), clipped_temp.filled(np.nan)
    # final_sci, final_temp = replace_clipped(clipped_sci, clipped_temp, np.ma.masked)
    final_sci = resampling_alberto(clipped_sci, np.nan)
    final_temp = resampling_alberto(clipped_temp, np.nan)
    plot_clipped_and_resampled(clipped_sci, final_sci, key, "Science", pdf)
    plot_clipped_and_resampled(clipped_temp, final_temp, key, "Template", pdf)
    pdf.close()
    to_table = [key, len(data_epochs[0]), stacked[0], stacked[1], stacked[2],
                clipped_sci, clipped_temp, final_sci, final_temp]
    final_table.add_row(to_table)

100% (18 of 18) |########################| Elapsed Time: 0:00:34 Time:  0:00:34


#### Saving final table

In [20]:
print(f'final_table has {len(final_table.colnames)} columns and {len(final_table)} rows')
table_name = f'{data_folder}/SIMBAD/SIMBAD_{class_name}_stacked_sigclipped_resampled.fits'
final_table.write(table_name, format='fits', overwrite=True)
print(exception_ids)

final_table has 9 columns and 18 rows
[]
